# Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

Part 3 of the assigment

#### Importing Libraries

In [2]:
import numpy as np #Numpy Library
import pandas as pd #Panda Library

In [3]:
from bs4 import BeautifulSoup #BeautifulSoup to read the html

In [4]:
import requests #to request the page

In [5]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
folium                    0.5.0                      py_0    conda-forge


In [6]:
!conda install -c conda-forge geocoder
import geocoder

Fetching package metadata .............
Solving package specifications: .

# All requested packages already installed.
# packages in environment at /opt/conda/envs/DSX-Python35:
#
geocoder                  1.38.1                     py_0    conda-forge


#### PART 1 of the assigment

In [7]:
source=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

In [8]:
soup = BeautifulSoup(source,'lxml')

In [9]:
tabla = soup.find('table', attrs={'class':'wikitable sortable'})
#print(tabla.prettify())  #prettify to display in a more friendly way

In [10]:
filas = tabla.find_all('tr')
l=[]
for tr in filas:
    td = tr.find_all('td')
    fila = [tr.text for tr in td]
    l.append(fila)
lista = pd.DataFrame(l, columns=['PostCode','Borough','Neigborhood'])

In [11]:
lista = lista.replace('\n','', regex=True)

In [12]:
lista.drop(lista.index[0],inplace=True)

In [13]:
lista = lista[lista.Borough != 'Not assigned']

In [14]:
lista['Neigborhood']= np.where(lista['Neigborhood']=='Not assigned',lista['Borough'],lista['Neigborhood'])

In [15]:
lista.sort_values('PostCode',inplace=True)

In [16]:
concatenar = {
    'Neigborhood': lambda x: ', '.join(x)
}

In [17]:
lista = lista.groupby(['PostCode','Borough'], as_index=False, sort=False).agg(concatenar)

## Part 2 of the assigment

Reading the csv file with the geospatial data for the postal codes that the geocoder returns error

In [18]:
gd=pd.read_csv('https://cocl.us/Geospatial_data',index_col=0)
print('Data loaded')

Data loaded


In [19]:
gd.head()

,Latitude,Longitude
Postal Code,,
M1B,43.806686,-79.194353
M1C,43.784535,-79.160497
M1E,43.763573,-79.188711
M1G,43.770992,-79.216917
M1H,43.773136,-79.239476


In [20]:
# The code was removed by Watson Studio for sharing.

Using the geocoder to obtain the latitude and longitude of the Postal Code

In [21]:
lista['Latitude']=0
lista['Longitude']=0
for i in range(0, len(lista)):
    ubicacion ='\''+lista['Borough'].iloc[i]+','+lista['PostCode'].iloc[i]+'\''
    try:
        g=geocoder.here(ubicacion,app_id=ID,app_code=CODE) # I'm using the geocoder HERE
        #print(i,"-",ubicacion,':',g.lat,' ',g.lng)
        lista['Latitude'].iloc[i] = g.lat
        lista['Longitude'].iloc[i]= g.lng
    except: #If the geocoder returns an error, I use gd dataframe to look for the latitude and longitude of the postal code
        postalcode=lista['PostCode'].iloc[i]
        lista['Latitude'].iloc[i] = gd['Latitude'].loc[postalcode]
        lista['Longitude'].iloc[i]= gd['Longitude'].loc[postalcode]
        #print(i,"-",ubicacion,':', lista['Latitude'].iloc[i],' ',lista['Longitude'].iloc[i])

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


# Part 3 of the assigment

In [22]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0   conda-forge
    geopy:         1.18.1-py_0 conda-forge

geographiclib- 100% |################################| Time: 0:00:00  21.62 MB/s
geopy-1.18.1-p 100% |################################| Time: 0:00:00  27.91 MB/s


In [23]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(lista['Borough'].unique()),
        lista.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


In [28]:
address = 'Toronto, CA'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/DSX-Python35/lib/python3.5/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Toronto are 43.7170226, -79.4197830350134.


In [29]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=13)

# add markers to map
for lat, lng, borough, neighborhood in zip(lista['Latitude'], lista['Longitude'], lista['Borough'], lista['Neigborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto